In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# forward dynamics

x_next, rhs_xdd, states, u, dt, q_min, q_max ,arm_p, _ , _, _  = rig_dyn.forward_dynamics()

In [3]:
# forces and moments at the base
F_at_base = rig_dyn.get_base_forces()
F_at_base_eval = cs.Function(
    'F_at_base', [arm_ss.q, arm_ss.q_dot, arm_ss.q_ddot, arm_ss.sim_p, arm_ss.gravity, arm_ss.base_T], [cs.vertcat(F_at_base[3:6], F_at_base[0:3])])

F_minus_xdd = F_at_base_eval(arm_ss.q, arm_ss.q_dot, rhs_xdd, arm_ss.sim_p, arm_ss.base_gravity, arm_ss.base_T)

In [4]:
F_next_ = cs.Function('Mnext', [states, u, arm_p, dt, q_min, q_max, arm_ss.gravity, arm_ss.base_gravity, arm_ss.base_T], [x_next, F_minus_xdd])
# F_next_.save("ARM_model.casadi")

In [5]:
q0 = [3.1, 0.7, 0.4, 2.1]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max, alpha.gravity, alpha.base_gravity, alpha.base_T0)
print(nextState)

[3.10047, 0.699907, 0.400129, 2.09984, 0.0136182, -0.00247992, 0.0037294, -0.00503415]


In [6]:
f_eval_base

DM([0.00863771, 0.0167685, -10.4963, -0.0143349, 0.968136, 0.000255273])

In [7]:
q0 = [3.1, 0.7, 0.4, 2.1]
q0dot = [0.0, 0.0, 0.0, 0.0]
u0 = [0.0, 0.0, 0.0, 0.0]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max, alpha.gravity, alpha.base_gravity, alpha.base_T0)
print(nextState)

[3.1, 0.699995, 0.400001, 2.1, -1.99705e-15, -0.00015522, 3.88695e-05, 0.000128295]


In [8]:
f_eval_base

DM([-0.000463945, 0.00426433, -10.5068, -0.0154247, 0.967756, 0.000394949])

In [9]:
# # c , cpp or matlab code generation for forward dynamics
# F_next_.generate("F_next_.c")
# os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0